# Get Run Data
Retrieve all records about runs.

In [1]:
import requests
from requests_oauthlib import OAuth2

import numpy as np
import pandas as pd

from utils import load_secrets

Load in tokens from secret file and use to establish OAuth 2.0

In [2]:
secrets = load_secrets()

In [3]:
auth = OAuth2(client_id=secrets["client_id"],
              token={
                  "access_token": secrets["access_token"]
              }
             )
auth

## Run Query
There is a Python FitBit API library, but it doesn't seem to be able to have functionality to access the specific endpoint we need - [Get Activity Logs List](https://dev.fitbit.com/build/reference/web-api/activity/#get-activity-logs-list)

Let's query for all runs since July 28th - the date I started training somewhat seriously.

In [4]:
API = "https://api.fitbit.com/1/user/-/activities/list.json"

params = {
    "afterDate": "2019-07-28",
    "sort": "asc",
    "limit": 20,  # This was max at time of writing
    "offset": 0  # Required for now, must be 0 (yes, dumb)
}

res = requests.get(API, params=params, auth=auth)

In [5]:
res.status_code

200

In [6]:
activities, pagination = res.json().values()
runs = [activity for activity in activities if activity["activityName"] == "Run"]

while pagination["next"] != "":
    print(f"Getting records {pagination['offset']}-{pagination['offset'] + pagination['offset']}")
    res = requests.get(pagination["next"], auth=auth)
    activities, pagination = res.json().values()
    runs.extend([activity for activity in activities if activity["activityName"] == "Run"])

Getting records 0-0
Getting records 20-40
Getting records 40-80
Getting records 60-120
Getting records 80-160
Getting records 100-200


In [7]:
len(runs)

26

In [8]:
def parse_run_record(record):
    series = pd.Series({
        "duration_sec": record["activeDuration"] / 1000,
        "distance": record["distance"] if "distance" in record else np.NaN,
        "date": record["startTime"],
        "pace": record["pace"] if "pace" in record else np.NaN,
        "speed": record["speed"] if "speed" in record else np.NaN,
        "steps": record["steps"],
        "calories": record["calories"],
        "waiting_sec": (record["duration"] - record["activeDuration"]) / 1000,
        "elevation_gain": record["elevationGain"]
    })
    series["duration_min"] = series["duration_sec"] / 60
    return series

In [9]:
series = [parse_run_record(run) for run in runs]
df = pd.DataFrame(series)

In [10]:
df

,duration_sec,distance,date,pace,speed,steps,calories,waiting_sec,elevation_gain,duration_min
0,2640.0,5.30381,2019-07-29T07:25:44.000-04:00,497.755387,7.232468,5406,459,6.0,24.079,44.000000
1,1927.0,5.02935,2019-08-01T14:55:39.000-04:00,383.150904,9.395776,4613,432,0.0,21.336,32.116667
2,2399.0,6.02178,2019-08-05T07:39:25.000-04:00,398.387188,9.036435,5753,515,0.0,23.774,39.983333
3,1790.0,5.01539,2019-08-06T08:28:33.000-04:00,356.901457,10.086818,4347,403,2.0,17.983,29.833333
4,1067.0,3.01078,2019-08-07T09:44:58.000-04:00,354.393214,10.158208,2620,207,124.0,9.144,17.783333
5,1984.0,5.02346,2019-08-09T10:11:15.000-04:00,394.946909,9.115149,4754,418,135.0,30.785,33.066667
6,1918.0,5.02006,2019-08-11T09:39:20.000-04:00,382.067147,9.422428,4646,372,154.0,50.902,31.966667
7,2221.0,6.02561,2019-08-13T08:44:11.000-04:00,368.593387,9.766860,5430,459,84.0,27.737,37.016667
8,1672.0,5.00162,2019-08-15T10:51:02.000-04:00,334.291689,10.769038,4105,352,155.0,26.518,27.866667
9,1712.0,5.00534,2019-08-18T11:52:10.000-04:00,342.034707,10.525248,4169,372,22.0,70.714,28.533333


In [11]:
def save_data(run_df):
    run_df.to_csv("data/runs.csv", index=None)
    run_df.to_parquet("data/runs.parquet", index=None, compression="gzip")

In [12]:
save_data(df)